# Esercizio 2 - Ottimizzazione della produzione di automobili con Gurobi

## Descrizione del problema

Una azienda automobilistica produce tre diversi modelli di autovettura:
- **Economica**
- **Normale**
- **Di lusso**

Ogni autovettura viene lavorata da tre robot: **A, B e C**. I tempi necessari alla lavorazione sono riportati nella tabella seguente, insieme al profitto netto realizzato per autovettura.

|           | Economica | Normale | Lusso |
|-----------|----------|---------|-------|
| **A**     | 20 min   | 30 min  | 62 min |
| **B**     | 31 min   | 42 min  | 51 min |
| **C**     | 16 min   | 81 min  | 10 min |
| **Prezzo**| 1000 €   | 1500 €  | 2200 € |

I robot hanno le seguenti disponibilità giornaliere:
- I robot **A e B** possono lavorare **8 ore al giorno** (480 minuti).
- Il robot **C** può lavorare **5 ore al giorno** (300 minuti).

## Vincoli del problema

1. Il numero di autovetture di **lusso** prodotte non deve superare il **20%** della produzione totale.
2. Il numero di autovetture **economiche** deve costituire **almeno il 40%** della produzione totale.
3. Le risorse disponibili dei robot **non devono essere superate**.

## Obiettivo

Determinare le quantità giornaliere da produrre per ciascun modello in modo tale da **massimizzare i profitti**, rispettando i vincoli di produzione.


In [ ]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('CarFactory')

# Car models
car_models = ['eco', 'std', 'lux']

# Robots
robots = ['A', 'B', 'C']

# Selling prices
prices = {'eco': 1000, 'std': 1500, 'lux': 2200}

# Working times (minutes)
times = {('A', 'eco'): 20, ('B', 'eco'): 31, ('C', 'eco'): 16,
         ('A', 'std'): 30, ('B', 'std'): 42, ('C', 'std'): 81,
         ('A', 'lux'): 62, ('B', 'lux'): 51, ('C', 'lux'): 10}

# Decision variables
x = m.addVars(robots, car_models, vtype=GRB.CONTINUOUS ,name='x')


# Objective function: maximize the profit
m.setObjective(sum(x[i,j]*prices[j] for i in robots for j in car_models), GRB.MAXIMIZE)

# Constraint: A,B <= 8h and C<= 5h
m.addConstrs(gp.quicksum(x[r, m] * times[r, m] for m in car_models) <= (8*60 if r in ['A', 'B'] else 5*60) for r in robots)

## Relational constrains
# Total cars production
total_cars = gp.quicksum(x[r, m] for r in robots for m in car_models)

# Constraint: luxury cars <= 20% of the total
m.addConstr(gp.quicksum(x[r, 'lux'] for r in robots) <= 0.2 * total_cars)

# Constraint: economy cars >= 40% of total
m.addConstr(gp.quicksum(x[r, 'eco'] for r in robots) >= 0.4 * total_cars)


# Solving
m.optimize()

# Output cleaning
if m.status == GRB.Status.OPTIMAL:
    print("Optimal solution:")
    for j in robots:
        for i in car_models:
            print(f"x[{j},{i}] = {x[j, i].X:.2f}")
    print(f"{m.ObjVal:.2f} $")
